In [6]:
import snap
from os import path

In [7]:
BASE_PATH = "../data/academia.stackexchange.com"
# FOLDED_USERID_TSV_PATH = path.join(BASE_PATH, "userid_folded_graph.tsv")
FOLDED_USERID_GRAPH_PATH = path.join(BASE_PATH, "Userid_Folded_Graph.graph")
# FOLDED_POSTID_TSV_PATH = path.join(BASE_PATH, "postid_folded_graph.tsv")

In [4]:
# user_graph = snap.LoadEdgeList(snap.PUNGraph, FOLDED_USERID_TSV_PATH, 0, 1)
# user_graph.GetNodes()
# user_graph.GetEdges()

137062264

In [12]:
f_in = snap.TFIn(FOLDED_USERID_GRAPH_PATH)
user_graph = snap.TUNGraph.Load(f_in)
print "nodes", user_graph.GetNodes()
print "edges", user_graph.GetEdges()

nodes 18653
edges 137062264
